This is where the metadata, feature space, and genres are read in and stored in the "songs" dictionary by `track_id`.

In [1]:
from pkg.spotifyclassifier import *
from pkg.graphs import *
from pkg.run import *
from pkg.wrangling import *
from pkg.constants import LABEL_LIST

In [2]:
default_song_data = FrozenMap(read_data('data/data.csv'))
print('Song data reading complete.')



Song data reading complete.


In [3]:
genres = FrozenMap(read_genres('data/data.csv'))
print('Genre data reading complete.')



Genre data reading complete.


In [4]:
default_feature_names = tuple(next(iter(default_song_data.values()))['features'].keys())

In [5]:
# Sanity check
l, dl = 0, 0
for x in default_song_data.values():
    if x['label'] == 1: l += 1
    else: dl += 1
print(' likes:', l, '\n', 'dislikes:', dl)

 likes: 1919 
 dislikes: 3789


In [6]:
test_cluster_size(default_song_data, 10)

2 2.2661993529803264e+16
3 1.0041686491846414e+16
4 3971861645771495.0
5 2864263083762424.5
6 2699856261343651.5
7 2129213427017124.0
8 1638328399972220.0
9 1538355294667852.5
10 1502268468566126.0


In [7]:
clustered_song_data, songs_by_cluster = {}, {}
clustered_song_data, songs_by_cluster = get_kmeans_clusters(default_song_data, NUM_CLUSTERS)

In [8]:
if set(next(iter(default_song_data.values()))['features'].keys()) == set(next(iter(clustered_song_data.values()))['features'].keys()):
    raise ValueError('Default features messed up.')

In [9]:
default_training_data, default_validation_data, clustered_training_data, clustered_validation_data = get_experiment_split(default_song_data, clustered_song_data)
training_clusters = get_train_clusters(clustered_training_data, songs_by_cluster[NUM_CLUSTERS])

In [10]:
print('----\nUnclustered\n----')
active_unclustered_results = run_active_suite(default_song_data, default_training_data, default_validation_data, SUPPORTED_ALGS, AL_STRATS)

----
Unclustered
----
101
svc accs w/ random
32.6	32.6	32.6	32.6	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	
101
svc accs w/ uncertainty
67.39	32.69	67.39	67.39	67.39	67.39	67.39	67.39	67.39	32.69	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.48	67.48	67.48	67.48	67.48	67.48	67.48	67.48	67.48	67.48	67.48	

/Users/adi/repos/spotify-music-discovery/venv/lib/python3.5/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


101
lsvc accs w/ random
67.04	66.95	66.08	66.95	60.12	50.74	67.39	66.43	67.39	37.51	39.26	32.6	67.39	45.22	61.43	67.39	67.39	67.39	32.6	67.39	45.31	67.39	46.27	40.31	52.67	33.39	47.15	67.39	32.6	45.57	32.6	67.39	32.6	32.6	32.6	58.63	67.39	66.78	46.36	67.39	64.94	67.39	33.04	32.6	67.39	65.81	67.39	45.66	32.6	61.43	54.86	59.59	67.39	32.6	60.21	64.32	32.6	45.13	32.6	37.24	67.39	67.39	56.44	67.39	32.6	67.39	32.6	54.95	50.21	32.77	59.5	32.6	65.29	59.33	67.39	64.76	67.39	67.39	67.39	41.01	67.39	46.71	67.39	67.39	59.5	67.39	46.53	67.39	59.15	67.39	65.46	32.6	61.61	55.3	33.04	59.85	54.16	40.92	61.08	40.75	32.6	
101
lsvc accs w/ uncertainty
45.92	45.92	45.92	43.03	32.6	67.39	67.39	67.39	38.47	32.6	67.39	37.33	67.39	65.46	43.38	54.33	55.65	32.6	67.39	55.3	64.76	32.6	67.39	67.39	32.6	32.6	56.79	59.77	60.47	32.6	51.44	60.73	67.39	67.39	32.6	45.66	52.05	45.39	50.74	54.25	56.96	36.28	67.39	65.29	67.39	32.6	39.17	44.69	67.39	67.39	67.39	64.94	67.39	53.72	32.6	61.08	59.59	67.39	67.39	58.54	61.7	67.39	

In [11]:
print('----\nClustered\n----')
active_clustered_results = run_active_suite(default_song_data, clustered_training_data, clustered_validation_data, SUPPORTED_ALGS, AL_STRATS)

----
Clustered
----
101
svc accs w/ random
67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	
101
svc accs w/ uncertainty
67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.39	67.3

In [12]:
print('\n----\nClustered w/ Cluster Sampling\n----')
active_cluster_sampled_results = run_clusters_suite(default_song_data, clustered_training_data, clustered_validation_data, training_clusters)


----
Clustered w/ Cluster Sampling
----
sgd accs w/ random
66.87	66.87	66.78	67.04	67.39	67.39	67.39	62.31	62.31	62.31	64.94	61.08	64.94	62.75	67.39	67.39	67.39	67.39	64.94	64.94	64.94	67.22	67.22	67.39	67.39	65.55	
sgd accs w/ uncertainty
34.53	33.21	67.39	67.39	66.6	67.39	67.39	65.99	66.08	66.34	67.39	67.39	67.39	66.69	66.87	66.95	67.39	67.39	67.39	66.17	65.55	65.38	66.25	66.25	66.6	65.55	


In [13]:
run_benchmarks_suite(
    default_song_data,
    default_training_data,
    default_validation_data,
    clustered_training_data,
    clustered_validation_data
)

svc accs:	0.6783523225241017	0.6783523225241017
lsvc accs:	0.6739702015775635	0.6739702015775635


/Users/adi/repos/spotify-music-discovery/venv/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


sgd accs:	0.6687116564417178	0.6722173531989483


In [15]:
result_list = [active_unclustered_results, active_clustered_results]

svc_results = get_new_results('svc', result_list, LABEL_LIST)
lsvc_results = get_new_results('lsvc', result_list, LABEL_LIST)
sgd_results = get_new_results('sgd', result_list, LABEL_LIST)

In [16]:
svc_y, svc_l = get_plottable_results(svc_results)
lsvc_y, lsvc_l = get_plottable_results(lsvc_results)
sgd_y, sgd_l = get_plottable_results(sgd_results)

svc_y_r, svc_l_r = svc_y[::2], svc_l[::2]
svc_y_u, svc_l_u = svc_y[1::2], svc_l[1::2]
# lsvc_y_r, lsvc_l_r = lsvc_y[::2], lsvc_l[::2]
# lsvc_y_u, lsvc_l_u = lsvc_y[1::2], lsvc_l[1::2]
sgd_y_r, sgd_l_r = sgd_y[::2], sgd_l[::2]
sgd_y_u, sgd_l_u = sgd_y[1::2], sgd_l[1::2]

In [17]:
# import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick

# accuracy_plot(svc_y_r, svc_l_r, 'SVC Random', svc_bench_acc)
# accuracy_plot(svc_y_u, svc_l_u, 'SVC Uncertainty', svc_bench_acc)
# accuracy_plot(lsvc_y, lsvc_l, 'LinearSVC', lsvc_bench_acc)
# accuracy_plot(sgd_y_r, sgd_l_r, 'SGD Random', sgd_bench_acc)
# accuracy_plot(sgd_y_u, sgd_l_u, 'SGD Uncertainty', sgd_bench_acc)

# # def get_accuracy_plots():
# #     for alg in SUPPORTED_ALGS:
# #         accuracy_plot(basic_results[i][:2], basic_titles[i])
        
# # get_accuracy_plots()


In [18]:
# result_list = [active_cluster_sampled_results]
# label_list = ['Clustered']
# sgd_c_results = get_new_results('sgd')
# sgd_c_y, sgd_c_l = get_plottable_results(sgd_c_results)

# # def accuracy_plot_c(y, labels, title, benchmark, save=False):
    
# #     # x = [i * (50//NUM_CLUSTERS) + 5 for i in range(len(y[0]))]
# #     # x = [i * (50//4) + 5 for i in range(len(y[0]))]
# #     x = [i + 5 for i in range(len(y[0]))]
 
# #     for i in range(len(y)):
# #         plt.plot(x, y[i], '--' if i % 2 == 0 else '-', label=labels[i])
# #     plt.plot(x, [benchmark for i in range(len(y[0]))], '-', label='Benchmark', color='black')
# #     plt.legend()
# #     plt.title(title)
# #     plt.xlabel('# Training Instances')
# #     plt.ylabel('Accuracy')
# #     plt.axis([5, 5+len(y[0]), 0, 1.0])
# #     plt.axes().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
# #     if save: plt.savefig('plots/' + str(NUM_CLUSTERS) + '_' + title.replace(' ', '_') + '.png')
# #     plt.show()

# accuracy_plot_c(sgd_c_y, sgd_c_l, 'SGD Cluster Sampling', sgd_bench_acc)